In [ ]:
# Import des libraries

import pandas as pd
from io import StringIO
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import fiona
import pyreadr
import os
import datetime
from shapely.geometry import Point
import xarray as xr

In [ ]:
# Récupération des données en sortie d'Arbocarto
print('Récupération des données en sortie d\'Arbocarto')

list_paths = ["C:\\Users\\aurel\\Documents\\ArbocartoDonnées\\2-2A.rda", "C:\\Users\\aurel\\Documents\\ArbocartoDonnées\\2-2B.rda"]
for i in range(1,96):
    if i ==20:
        continue
    if i < 10:
        ch = f"C:\\Users\\aurel\\Documents\\ArbocartoDonnées\\2-0{i}.rda"
    else:
        ch = f"C:\\Users\\aurel\\Documents\\ArbocartoDonnées\\2-{i}.rda"
    list_paths.append(ch)

list_map = []

list_map.extend([pyreadr.read_r(path)["trajectory"].copy() for path in list_paths])

print(list_map)

# for i in range(len(list_paths)):
#    dict = pyreadr.read_r(list_paths[i])
#    print(dict.keys())
#    list_map.append(dict["trajectory"].copy())
#    print(list_map)

In [ ]:
# Fusion des données d'Arbocarto sur le premier jour (Test)

pd.set_option('display.max_rows', None)
justsee = [df[df['DATE'] == datetime.date(2021,1,1)] for df in list_map]

In [ ]:
mapjustsee = pd.concat(justsee, ignore_index=True)
print(mapjustsee.head())

In [ ]:
# Fusion des données d'Arbocarto au premier mois de l'été

start_date = datetime.date(2022,6,21)
end_date = datetime.date(2022,7,20)

In [ ]:
summermap = [df[(df['DATE'] >= start_date) & (df['DATE'] <= end_date)] for df in list_map]

In [ ]:
map = pd.concat(summermap, ignore_index=True)

In [ ]:
print(map.head())

In [ ]:
# Récupération du découpage IRIS-GE
print('Récupération du découpage IRIS-GE')

import os
irisge = r"C:\Users\aurel\Downloads\IRIS-GE_3-0__GPKG_LAMB93_FXX_2025-01-01\IRIS-GE_3-0__GPKG_LAMB93_FXX_2025-01-01\IRIS-GE\1_DONNEES_LIVRAISON_2025-06-00081\IRIS-GE_3-0_GPKG_LAMB93_FXX-ED2025-01-01\iris.gpkg"
print(os.path.exists(irisge))
import fiona
layers = fiona.listlayers(irisge)
print("Layers:", layers)

In [ ]:
import geopandas as gpd

irisge = r"C:\Users\aurel\Downloads\IRIS-GE_3-0__GPKG_LAMB93_FXX_2025-01-01\IRIS-GE_3-0__GPKG_LAMB93_FXX_2025-01-01\IRIS-GE\1_DONNEES_LIVRAISON_2025-06-00081\IRIS-GE_3-0_GPKG_LAMB93_FXX-ED2025-01-01\iris.gpkg"

irismap = gpd.read_file(irisge, layer="iris")
print(irismap.head())

In [ ]:
# Correspondance entre les données d'Arbocarto et IRIS-GE

# Check for matches with code_insee
matches_insee = irismap['code_insee'].isin(map['ID'])
print("Number of matches with code_insee:", matches_insee.sum())

# Check for matches with code_iris
matches_iris = irismap['code_iris'].isin(map['ID'])
print("Number of matches with code_iris:", matches_iris.sum())

# Optionally, see which values match
matching_insee_values = irismap.loc[matches_insee, 'code_insee']
matching_iris_values = irismap.loc[matches_iris, 'code_iris']

print("Matching code_insee values:", matching_insee_values.tolist())
print("Matching code_iris values:", matching_iris_values.tolist())

In [ ]:
print(len(map[map['DATE'] == datetime.date(2022, 6, 21)]))
print(len(irismap))

In [ ]:
# irisgdf = gpd.GeoDataFrame(irismap, geometry=gpd.points_from_xy(irismap['geometry'].x, irismap['geometry'].y))
irismap.plot(column='code_iris', cmap='viridis', legend=True)
plt.show()

In [ ]:
irismap_sample = irismap.sample(500, random_state=1)
irismap_sample.plot(column='code_iris', cmap='viridis', legend=True)
plt.show()

In [ ]:
# Représentation des moustiques adultes le premier jour de l'été

# 1. Filter your map DataFrame for the 21st of June
date_to_plot = datetime.date(2022, 6, 21)
map_june21 = map[map['DATE'] == date_to_plot]

# 2. Merge with irismap to get the geometry
map_plot = pd.merge(map_june21, irismap, left_on='ID', right_on='code_iris', how='left')

# 3. Convert to GeoDataFrame (ensure geometry column is present)
map_plot = gpd.GeoDataFrame(map_plot, geometry='geometry', crs=irismap.crs)

# 4. Plot the Ahm column
map_plot.plot(column='Ahm', figsize=(10, 6), legend=True, cmap='viridis')
plt.title(f"Ahm on {date_to_plot}")
plt.axis('off')
plt.show()

In [ ]:
# Pourquoi c'est vide par endroits ?

print(map_plot[['ID', 'code_iris', 'Ahm']].isna().sum())
print(map_plot[map_plot['Ahm'].isna()][['ID', 'code_iris']])

In [ ]:
print(map['ID'].dtype, irismap['code_iris'].dtype)
print(map['ID'].head(), irismap['code_iris'].head())

In [ ]:
corine = r"C:\Users\aurel\Downloads\corine\CLC_PNE_RG\CORINE_LAND_COVER_FRANCE_METROPOLITAINE_EPSG2154.gpkg"
layers = fiona.listlayers(corine)
print("Layers:", layers)

In [ ]:
corinemap = gpd.read_file(corine, layer="CORINE_LAND_COVER_FRANCE_METROPOLITAINE_EPSG2154")
print(corinemap.head())

In [ ]:
# Dissoudre les polygones IRIS par code_iris (ou un autre identifiant pertinent)
mapplotdiss = map_plot.dissolve(by='code_iris')

# Dissoudre les polygones CORINE par code_18 (ou un autre identifiant pertinent)
corinemapdiss = corinemap.dissolve(by='code_18')

mapplotdiss['geometry'] = mapplotdiss.geometry.simplify(20)      # 20 mètres de tolérance (à ajuster)
corinemapdiss['geometry'] = corinemapdiss.geometry.simplify(20)

# Faire l'overlay sur les couches dissoutes
maplandcover = gpd.overlay(mapplotdiss, corinemapdiss, how="identity")

In [ ]:
# Ensure both GeoDataFrames use the same CRS
corinemap = corinemap.to_crs(map_plot.crs)

maplandcover = gpd.overlay(map_plot, corinemap, how="intersection")

pd.set_option('display.max_columns', None)
print(maplandcover.head())

In [ ]:
maplandcover = gpd.GeoDataFrame(maplandcover, geometry='geometry', crs=irismap.crs)

import matplotlib.pyplot as plt
import numpy as np

# Convert code_18 to numeric if not already
maplandcover['code_18_num'] = pd.to_numeric(maplandcover['code_18'], errors='coerce')

# Create quantile bins (e.g., 10 quantiles)
maplandcover['code_18_quantile'] = pd.qcut(maplandcover['code_18_num'], q=10, labels=False, duplicates='drop')

# Plot with a continuous colormap and colorbar
fig, ax = plt.subplots(figsize=(10, 6))
maplandcover.plot(
    column='code_18_quantile',
    ax=ax,
    cmap='viridis',
    legend=True,
    legend_kwds={'label': "Land cover quantile", 'shrink': 0.7}
)
plt.title("Land cover quantiles (continuous legend)")
plt.axis('off')
plt.show()

# maplandcover.plot(column='code_18', figsize=(10, 6), legend=True, cmap='Greens')
# plt.title(f"Land cover on {date_to_plot}")
# plt.axis('off')
# plt.show()

In [ ]:
print(type(maplandcover))
maplandcover = gpd.GeoDataFrame(maplandcover, geometry='geometry', crs=irismap.crs)

In [ ]:
# Exemple : ne prendre que les polygones IRIS et CORINE d'un département
irismap_subset = irismap[irismap['code_insee'].str.startswith('75')]  # Paris
corinemap_subset = corinemap[corinemap.intersects(irismap_subset.unary_union)]

# Dissoudre
iris_diss = irismap_subset.dissolve(by='code_iris')
corine_diss = corinemap_subset.dissolve(by='code_18')

# Overlay
result = gpd.overlay(iris_diss, corine_diss, how="identity")
print(result.head())

In [ ]:
result.plot(column='code_18', figsize=(10, 6), legend=True, cmap='viridis')
plt.show()

In [ ]:
comparison = gpd.overlay(iris_diss, corine_diss, how="intersection")
comparison.plot(column='code_18', figsize=(10, 6), legend=True, cmap='viridis')
plt.show()

In [ ]:
maplandcover = gpd.overlay(map_plot, corinemap, how="identity")

In [ ]:
print(maplandcover.head())

In [ ]:
maplandcover.plot(column='code_18', figsize=(10, 6), legend=True, cmap='viridis')
plt.show()

In [ ]:
# Vérifie que le CRS est en mètres (EPSG:2154 pour la France)
if corinemap.crs.to_epsg() != 2154:
    corinemap = corinemap.to_crs(epsg=2154)

# Surface médiane des polygones (en m²)
areas = irismap.geometry.area
print("Surface médiane des polygones CORINE (m²):", np.median(areas))
print("Surface minimale des polygones CORINE (m²):", areas.min())

# Espacement médian entre centroïdes (en mètres)
from scipy.spatial import cKDTree
centroids = np.array([(geom.centroid.x, geom.centroid.y) for geom in corinemap.geometry])
tree = cKDTree(centroids)
distances, _ = tree.query(centroids, k=2)
min_distances = distances[:, 1]
print("Espacement médian entre centroïdes CORINE (m):", np.median(min_distances))

In [ ]:
print(areas.max())

Ici commence l'intégration des données météo

In [ ]:
# import cdsapi

# dataset = "insitu-gridded-observations-europe"
# request = {
#     "product_type": "ensemble_spread",
#     "variable": [
#         "mean_temperature",
#         "minimum_temperature",
#         "maximum_temperature",
#         "precipitation_amount",
#         "sea_level_pressure",
#         "surface_shortwave_downwelling_radiation",
#         "relative_humidity"
#     ],
#     "grid_resolution": "0_1deg",
#     "period": "2011_2023",
#     "version": ["29_0e"]
# }

# client = cdsapi.Client()
# client.retrieve(dataset, request).download()


In [ ]:
nc_dir = r"C:\Users\aurel\Downloads\copernicusmeteo1"

# List all .nc files in the directory
nc_files = [os.path.join(nc_dir, f) for f in os.listdir(nc_dir) if f.endswith('.nc')]

hu = xr.open_dataset(nc_files[0])
pp = xr.open_dataset(nc_files[1])
qq = xr.open_dataset(nc_files[2])
rr = xr.open_dataset(nc_files[3])
tg = xr.open_dataset(nc_files[4])
tn = xr.open_dataset(nc_files[5])
tx = xr.open_dataset(nc_files[6])

print(hu, pp, qq, rr, tg, tn, tx)

In [ ]:
# Prendre un des fichiers, par exemple 'hu'
lats = hu.latitude.values
lons = hu.longitude.values

# Calculer la résolution (en degrés)
lat_res = np.abs(lats[1] - lats[0])
lon_res = np.abs(lons[1] - lons[0])
print(f"Résolution latitude : {lat_res}°")
print(f"Résolution longitude : {lon_res}°")

# Optionnel : convertir en kilomètres (approximation)
# 1° latitude ≈ 111 km ; 1° longitude ≈ 111 km * cos(latitude)
mean_lat = np.mean(lats)
lat_km = lat_res * 111
lon_km = lon_res * 111 * np.cos(np.deg2rad(mean_lat))
print(f"Résolution ≈ {lat_km:.1f} km (lat) x {lon_km:.1f} km (lon)")

In [ ]:
# Définir la période et la région d'intérêt
date_debut = "2022-06-21"
date_fin = "2022-09-20"
lat_min, lat_max = 41.0, 52.0   # exemple : France métropolitaine
lon_min, lon_max = -5.0, 10.0

# Fonction pour extraire et filtrer une variable
def extract_var(ds, varname):
    # Sélectionner la période et la région
    da = ds[varname].sel(
        time=slice(date_debut, date_fin),
        latitude=slice(lat_min, lat_max),
        longitude=slice(lon_min, lon_max)
    )
    # Conversion en DataFrame et suppression des NaN
    df = da.to_dataframe().reset_index().dropna(subset=[varname])
    return df

# Extraction pour chaque variable
df_hu = extract_var(hu, 'hu')
df_pp = extract_var(pp, 'pp')
df_qq = extract_var(qq, 'qq')
df_rr = extract_var(rr, 'rr')
df_tg = extract_var(tg, 'tg')
df_tn = extract_var(tn, 'tn')
df_tx = extract_var(tx, 'tx')

# Fusion sur les colonnes communes (time, latitude, longitude)
variables = [df_hu, df_pp, df_qq, df_rr, df_tg, df_tn, df_tx]
from functools import reduce
cartemeteo = reduce(lambda left, right: pd.merge(left, right, on=['time', 'latitude', 'longitude'], how='outer'), variables)

print(cartemeteo.info())
print(cartemeteo.head())

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Création de la colonne geometry
cartemeteo['geometry'] = [Point(xy) for xy in zip(cartemeteo['longitude'], cartemeteo['latitude'])]

# Conversion en GeoDataFrame (CRS WGS84 par défaut, adapte si besoin)
gdf_points = gpd.GeoDataFrame(cartemeteo, geometry='geometry', crs="EPSG:4326")

# Exemple d’affichage de l’humidité relative pour une date donnée
date_to_plot = pd.Timestamp("2022-06-21")
gdf_plot = gdf_points[gdf_points['time'] == date_to_plot]

if gdf_plot.empty:
    print("Aucune donnée pour la date sélectionnée.")
else:
    gdf_plot.plot(column='hu', legend=True, cmap='Blues', figsize=(10, 6))
    plt.title("Points météo - humidité relative (hu)")
    plt.show()

In [ ]:
cartemeteo['time'] = pd.to_datetime(cartemeteo['time'])
print(cartemeteo['time'])

In [ ]:
gdf_points = gdf_points.to_crs(maplandcover.crs)
print(gdf_points.crs == maplandcover.crs)

In [ ]:
# Jointure spatiale : chaque point météo reçoit l'index du polygone le contenant
points_in_poly = gpd.sjoin(gdf_plot, maplandcover, how='left', predicate='within')
print(points_in_poly.head())

In [ ]:
# Compte du nombre de points météo (par exemple 'hu') par polygone
count_points = points_in_poly.groupby('index_right').size()
print(count_points)

In [ ]:
# Tous les index de polygones
all_polygons = set(maplandcover.index)
# Ceux qui ont au moins un point
polygons_with_points = set(count_points.index)
# Ceux qui n'ont aucun point
polygons_without_points = all_polygons - polygons_with_points

print(f"Polygones avec au moins un point météo : {len(polygons_with_points)}")
print(f"Polygones sans aucun point météo : {len(polygons_without_points)}")

In [ ]:
print(f"Polygones avec plus d'un point météo : {(count_points > 1).sum()}")

In [ ]:
import numpy as np
from shapely.geometry import Point
from scipy.spatial import cKDTree

# Récupérer les centroïdes des polygones
centroids = np.array([(geom.centroid.x, geom.centroid.y) for geom in maplandcover.geometry])

# Construire un arbre de recherche rapide
tree = cKDTree(centroids)

# Pour chaque centroïde, trouver la distance au centroïde le plus proche (hors lui-même)
distances, _ = tree.query(centroids, k=2)  # k=2 pour inclure soi-même et le plus proche
min_distances = distances[:, 1]  # On prend la 2e colonne (le plus proche hors soi)

# Espacement minimum, maximum, médian
print(f"Espacement minimum entre polygones : {min_distances.min():.2f} unités")
print(f"Espacement médian entre polygones : {np.median(min_distances):.2f} unités")
print(f"Espacement maximum entre polygones : {min_distances.max():.2f} unités")

In [ ]:
# Surface de chaque polygone (en m² si CRS en mètres)
areas = maplandcover.geometry.area
print("Surface médiane des polygones (m²):", np.median(areas))
print("Surface minimale des polygones (m²):", areas.min())